# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

The _actual sequences_ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The _sequencing reads_ (located in the FASTQ file) are from the reverse end of these actual sequences, so the first thing in the sequencing reads is the reverse complement of the barcode followed by the reverse complement of the constant sequence, etc.
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [62]:
# HW 4 Prob 1
# Completed with the help of Theresa Chen and Sophie Kogut, as well as lecture 8 and 9 Jupyter notebooks

# Import necessary packages
import re
import Bio.SeqIO


# Defining a function to produce the reverse complement of a sequence, taken from the lecture 8 Jupyter notebook
def reverse_complement(seq):
    rc_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'}
    revcomplement = []
    for nt in reversed(seq.upper()):
        revcomplement.append(rc_dict[nt])
    return ''.join(revcomplement)

# Converting the sequence part of each read into a string, taken from the lecture 9 Jupyter notebook
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))
seqreads_str = []
for seqrecord in seqreads:
    seqreads_str.append(str(seqrecord.seq))
    # Now all sequences are in a list called seqreads_str

# All of the following code in this cell is my own

# Defining the end of NA and HA sequences
end_NA = 'CACGATAGATAAATAATAGTGCACCAT'
end_HA = 'CCGGATTTGCATATAATGATGCACCAT'

# Creating seperate lists for NA and HA sequences
NA_seq_list = []
HA_seq_list = []

# Defining a search pattern for HA and NA sequences
NA_search = re.compile(end_NA)
HA_search = re.compile(end_HA)

for read in seqreads_str:   
    UCread = read.upper()
    rev_read = reverse_complement(UCread)     
    match_NA = NA_search.search(rev_read)   # Finding sequences in the seqreads_str list that contain HA or NA
    match_HA = HA_search.search(rev_read)
    if match_NA:                            # If a sequence contains NA, it's appended to the NA_seq_list
        NA_seq_list.append(read)
    if match_HA:                            # And vice versa for HA sequences and the HA_seq_list
        HA_seq_list.append(read)


print(f"{len(NA_seq_list)} reads map to NA, {len(HA_seq_list)} reads map to HA")



4122 reads map to NA, 5409 reads map to HA


2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [61]:
# HW 4 Prob 2
# Defining a function to read a barcode of a specific length, add it to a dictionary, and count the occurances of said barcode
# Structure taken from the lecture 9 Jupyter notebook, edited to solve this problem
def count_barcodes(seqs, bclen=16, upstream='AGGCGGCCGC'):
    """Parse and count barcodes.
    
    Parameters
    ----------
    seqs : list
        DNA sequences.
    bclen : int
        Length of barcode
    upstream : str
        Sequence upstream of barcode.
    downstream : str
        Sequence downstream of barcode.
        
    Returns
    -------
    dict
        Keyed by each valid barcode, value is number of times the barcode
        is observed.
        
    Note
    ----
    The function is **not** case-sensitive, and all barcodes are reported
    in upper-case.
    
    """
    
    # compile our re
    barcode_re = re.compile(upstream + f"(?P<barcode>[ATCGN]{{{bclen}}})$")
    
    # Create a dictionary for the barcodes and their counts
    barcode_counts = {} 
    
    for seq in seqs:
        seq = seq.upper()
        seq = reverse_complement(seq)
        match = barcode_re.search(seq)  # Searching for sequences with the 'upstream' and a 'barcode'
        if match:
            barcode = match.group("barcode")    # barcode is equal to the 16 nt <barcode> in the search pattern
            if barcode in barcode_counts:
                barcode_counts[barcode] += 1    # If the barcode is in the dictionary, a value of 1 is added to the count
            else:
                barcode_counts[barcode] = 1     # If the barcode is not already in the dictionary, it is created as a key with the value 1
        
    return barcode_counts

# The rest of the code in this cell is my own

# Running the count_barcodes function on the lists of NA and HA sequences
# Now I have 2 dictionaries with barcodes and their counts
NA_counts = count_barcodes(seqs = NA_seq_list)
HA_counts = count_barcodes(seqs = HA_seq_list)

# Finding the NA barcode with the most counts
def most_counts(count_dict):
    highest_count_int = 0                           # Creating a value that keeps track of the highest barcode count
    highest_count_str = ''                          # Creating a string to keep track of the string with the highest barcode count
    for bc in count_dict:                           # The function goes through all of the barcodes in the dictionary
        if count_dict[bc] > highest_count_int:      # If the count of a barcode in the dictionary is higher than the highest barcode count so far, it gets replaced by the higher integer
            highest_count_int = count_dict[bc]      
            highest_count_str = bc                  # If the barcode count of a barcode is higher than the highest barcode count so far, the barcode replaces the highest barcode string
    return highest_count_str                        # The function returns the string with the highest barcode count

# Renaming the string that comes back with the highest barcode count from each dictionary
highest_NA_str = most_counts(count_dict = NA_counts)
highest_HA_str = most_counts(count_dict = HA_counts)

print(f"The NA barcode with the most counts is {highest_NA_str}, which has {NA_counts[highest_NA_str]} counts")
print(f"The HA barcode with the most counts is {highest_HA_str}, which has {HA_counts[highest_HA_str]} counts")

The NA barcode with the most counts is ACCAGTTCTCCCCGGG, which has 152 counts
The HA barcode with the most counts is CCCGACCCGACATTAA, which has 155 counts
